# Internet of Things (2IMN25)
### Machine Learning Assignment

### Introduction

The objective of the assignment is to use supervised machine learning techniques to make predictions about the net energy consumed based on the weather conditions. This will be accomplished by training a regression model on (real) data obtained from the weather sensor and energy meter over this year. This model will then be used to make predictions about the net energy consumption for data-points not *seen* by the model during training.

In [1]:
""" Import all libraries here """
import numpy as np
import os
import math
from datetime import datetime
import pandas as pd
from dateutil import parser
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler


## Reading the dataset

In the code box below, please read in the data-sets into suitable data structures. The key task here would be to contruct the feature vectors which would be fed to the regression algorithm.

The datasets for the weather, rain and energy consumption have been provided in the zip file, extract this zip file into the same folder as the python notebook. 

*Hint : Be sure to check if there are any missing fields in the provided data*

In [2]:
def merge_weather_rain_data(data_path):
    """ 
    Reads in the CSV files for weather and rain data
    Merged them into a single array based on matching time-stamps
    
    """
        
    """ Read in the weather"""
    weather_array = pd.read_csv(os.path.join(data_path,'Weather-Eindhoven.csv'),
                             delimiter = ',',
                             header=0,
                             parse_dates = [0],
                             usecols = [0,2,4,6,7,8]) #Drop "chill", too many missing values
    weather_array  = weather_array.as_matrix()

    """ Read in the rain data"""
    rain_array = pd.read_csv(os.path.join(data_path,'Rain-Best.csv'),
                             delimiter = ',',
                             header=0,
                             parse_dates = [0])

    rain_array = rain_array.as_matrix()

    """ Synchronize the 2 data-sets by removing the extra data from the rainfall dataset"""
    remove_rows = []
    for element,idx in zip(rain_array[:,0],range(len(rain_array[:,0]))):
        if element.minute%10 != 0:
            remove_rows.append(idx)

    synced_rain_array = np.delete(rain_array,remove_rows,axis = 0)

    print(synced_rain_array.shape)
    print(weather_array[:,0].shape)

    """ Merge the weather and rain data when time-stamps are equal """
    merged_array = []
    for rainElem in synced_rain_array:
        try :
            merged_row = []
            valid_row = list(weather_array[:,0]).index(rainElem[0])
            for weatherFeature in weather_array[valid_row]:
                merged_row.append(weatherFeature)
            merged_row.append(rainElem[1])
            merged_array.append(np.asarray(merged_row))
        except ValueError:
            print('Did not find matching timestamp in the weather data, continuing')
            continue

    merged_array = np.asarray(merged_array)
    return merged_array

# Create the merged array
data_path = os.path.join(os.getcwd(),'data')
merged_array = merge_weather_rain_data(data_path)

(43844, 2)
(43872,)
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matchin

In [4]:

def clean_up(merged_array):
    """ 
    Clean up the data by removing rows with missing data points 
    
    """
    missing_data_rows = []
    for row,idx in zip(merged_array,range(merged_array.shape[0])):
        if row[0].month > 10:
            missing_data_rows.append(idx)
            continue # No energy data for the month of November so delete those too.
        for elem in row[1:]: #Timestamp cannot be "checked"
            if math.isnan(elem):
                missing_data_rows.append(idx)
                break # To prevent the same row from being added multiple times to the list, break when the first "nan" is found
    clean_data_array = np.delete(merged_array,missing_data_rows,axis=0)
    return clean_data_array

clean_data_array = clean_up(merged_array)


In [6]:
def create_dataframe(energyDir):
    """ Returns a dataframe that is formed by concatenating all the energy data files """
    fileList = []
    energyDir = os.path.join(os.getcwd(),energyDir)
    dirList = [os.path.join(energyDir, o) for o in os.listdir(energyDir) 
                    if os.path.isdir(os.path.join(energyDir,o))]
    
    sortedDirList = sorted(dirList)
    for direc in sortedDirList:
        dirFiles = [os.path.join(direc, f) for f in os.listdir(direc) 
                    if os.path.isfile(os.path.join(direc,f))]
        for f in dirFiles:
            fileList.append(f)
        
    fileList = sorted(fileList)
    frame = pd.DataFrame()
    frameList = []
    for f in fileList:
        try:
            df = pd.read_csv(f,index_col=None, header=None,parse_dates=[[0,1]],skipfooter=2)
            frameList.append(df)
        except:
            continue
    frame = pd.concat(frameList)
    sparseFrame = frame.drop([2,3,4,5,6,7,8,9,13],axis=1)
    sparseFrame.columns = ['TimeStamp','Energy Consumed','Energy Produced','Solar Energy Produced']
    return sparseFrame
    
energyFrame = create_dataframe('data/2017')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


               TimeStamp  Energy Consumed  Energy Produced  \
0    2017-01-01 00:00:02              420                0   
1    2017-01-01 00:00:12              420                0   
2    2017-01-01 00:00:22              420                0   
3    2017-01-01 00:00:32              420                0   
4    2017-01-01 00:00:42              420                0   
5    2017-01-01 00:00:52              420                0   
6    2017-01-01 00:01:02              420                0   
7    2017-01-01 00:01:12              420                0   
8    2017-01-01 00:01:22              530                0   
9    2017-01-01 00:01:32              520                0   
10   2017-01-01 00:01:42              530                0   
11   2017-01-01 00:01:52              590                0   
12   2017-01-01 00:02:02              520                0   
13   2017-01-01 00:02:12              520                0   
14   2017-01-01 00:02:22              520                0   
15   201

In [ ]:
def energy_lookup(data_array,energyFrame,idx):
    """ 
    Given the index, calculated the net-energy consumed in the 10-minute interval preceeding current time
    
    """
    try:
        subFrame = energyFrame[(energyFrame['TimeStamp']>data_array[idx-1][0]) & (energyFrame['TimeStamp']<data_array[idx][0])]
        if subFrame.shape[0] == 0:
            return None,None
        energyList = []
        solarList = []
        for cons,prod,solar in zip(subFrame['Energy Consumed'],subFrame['Energy Produced'],subFrame['Solar Energy Produced']):
            energyList.append(cons-prod)
            solarList.append(solar)
        return np.sum(np.asarray(energyList)),np.sum(np.asarray(solar))
    except:
        return None,None

def create_labels(data_array,energyFrame):
    """
    Uses the look-up function to create labels (if possible)
    for every feature vector in the feature array.
    
    """
    
    missing_labels = []
    labels = []
    labels_solar = []
    labels.append(0)
    labels_solar.append(0)
    energyFrame['TimeStamp'] = pd.to_datetime(energyFrame['TimeStamp']) 
    for row,idx in zip(data_array,range(data_array.shape[0])):
        if idx == 0:
            continue
        netEnergy,netSolar = energy_lookup(data_array,energyFrame,idx)
        if netEnergy is None and netSolar is None:
            print('Missing label for row {}'.format(idx))
            missing_labels.append(idx)
        else:
            print('{} {}'.format(row[0],netEnergy,netSolar))
            labels.append(netEnergy)
            labels_solar.append(netSolar)
    return labels,labels_solar,missing_labels

labels,labels_solar,missing_labels = create_labels(clean_data_array,energyFrame)
        
            

Missing label for row 1
Missing label for row 2
Missing label for row 3
Missing label for row 4
Missing label for row 5
Missing label for row 6
Missing label for row 7
Missing label for row 8
Missing label for row 9
Missing label for row 10
Missing label for row 11
Missing label for row 12
Missing label for row 13
Missing label for row 14
Missing label for row 15
Missing label for row 16
Missing label for row 17
Missing label for row 18
Missing label for row 19
Missing label for row 20
Missing label for row 21
Missing label for row 22
Missing label for row 23
Missing label for row 24
Missing label for row 25
Missing label for row 26
Missing label for row 27
Missing label for row 28
Missing label for row 29
Missing label for row 30
Missing label for row 31
Missing label for row 32
Missing label for row 33
Missing label for row 34
Missing label for row 35
Missing label for row 36
Missing label for row 37
Missing label for row 38
Missing label for row 39
Missing label for row 40
Missing l

Missing label for row 323
Missing label for row 324
Missing label for row 325
Missing label for row 326
Missing label for row 327
Missing label for row 328
Missing label for row 329
Missing label for row 330
Missing label for row 331
Missing label for row 332
Missing label for row 333
Missing label for row 334
Missing label for row 335
Missing label for row 336
Missing label for row 337
Missing label for row 338
Missing label for row 339
Missing label for row 340
Missing label for row 341
Missing label for row 342
Missing label for row 343
Missing label for row 344
Missing label for row 345
Missing label for row 346
Missing label for row 347
Missing label for row 348
Missing label for row 349
Missing label for row 350
Missing label for row 351
Missing label for row 352
Missing label for row 353
Missing label for row 354
Missing label for row 355
Missing label for row 356
Missing label for row 357
Missing label for row 358
Missing label for row 359
Missing label for row 360
Missing labe

Missing label for row 639
Missing label for row 640
Missing label for row 641
Missing label for row 642
Missing label for row 643
Missing label for row 644
Missing label for row 645
Missing label for row 646
Missing label for row 647
Missing label for row 648
Missing label for row 649
Missing label for row 650
Missing label for row 651
Missing label for row 652
Missing label for row 653
Missing label for row 654
Missing label for row 655
Missing label for row 656
Missing label for row 657
Missing label for row 658
Missing label for row 659
Missing label for row 660
Missing label for row 661
Missing label for row 662
Missing label for row 663
Missing label for row 664
Missing label for row 665
Missing label for row 666
Missing label for row 667
Missing label for row 668
Missing label for row 669
Missing label for row 670
Missing label for row 671
Missing label for row 672
Missing label for row 673
Missing label for row 674
Missing label for row 675
Missing label for row 676
Missing labe

Missing label for row 965
Missing label for row 966
Missing label for row 967
Missing label for row 968
Missing label for row 969
Missing label for row 970
Missing label for row 971
Missing label for row 972
Missing label for row 973
Missing label for row 974
Missing label for row 975
Missing label for row 976
Missing label for row 977
Missing label for row 978
Missing label for row 979
Missing label for row 980
Missing label for row 981
Missing label for row 982
Missing label for row 983
Missing label for row 984
Missing label for row 985
Missing label for row 986
Missing label for row 987
Missing label for row 988
Missing label for row 989
Missing label for row 990
Missing label for row 991
Missing label for row 992
Missing label for row 993
Missing label for row 994
Missing label for row 995
Missing label for row 996
Missing label for row 997
Missing label for row 998
Missing label for row 999
Missing label for row 1000
Missing label for row 1001
Missing label for row 1002
Missing l

Missing label for row 1272
Missing label for row 1273
Missing label for row 1274
Missing label for row 1275
Missing label for row 1276
Missing label for row 1277
Missing label for row 1278
Missing label for row 1279
Missing label for row 1280
Missing label for row 1281
Missing label for row 1282
Missing label for row 1283
Missing label for row 1284
Missing label for row 1285
Missing label for row 1286
Missing label for row 1287
Missing label for row 1288
Missing label for row 1289
Missing label for row 1290
Missing label for row 1291
Missing label for row 1292
Missing label for row 1293
Missing label for row 1294
Missing label for row 1295
Missing label for row 1296
Missing label for row 1297
Missing label for row 1298
Missing label for row 1299
Missing label for row 1300
Missing label for row 1301
Missing label for row 1302
Missing label for row 1303
Missing label for row 1304
Missing label for row 1305
Missing label for row 1306
Missing label for row 1307
Missing label for row 1308
M

...

Missing label for row 19851
Missing label for row 19852
Missing label for row 19853
Missing label for row 19854
Missing label for row 19855
Missing label for row 19856
Missing label for row 19857
Missing label for row 19858
Missing label for row 19859
Missing label for row 19860
Missing label for row 19861
Missing label for row 19862
Missing label for row 19863
Missing label for row 19864
Missing label for row 19865
Missing label for row 19866
Missing label for row 19867
Missing label for row 19868
Missing label for row 19869
Missing label for row 19870
Missing label for row 19871
Missing label for row 19872
Missing label for row 19873
Missing label for row 19874
Missing label for row 19875
Missing label for row 19876
Missing label for row 19877
Missing label for row 19878
Missing label for row 19879
Missing label for row 19880
Missing label for row 19881
Missing label for row 19882
Missing label for row 19883
Missing label for row 19884
Missing label for row 19885
Missing label for ro

Missing label for row 20150
Missing label for row 20151
Missing label for row 20152
Missing label for row 20153
Missing label for row 20154
Missing label for row 20155
Missing label for row 20156
Missing label for row 20157
Missing label for row 20158
Missing label for row 20159
Missing label for row 20160
Missing label for row 20161
Missing label for row 20162
Missing label for row 20163
Missing label for row 20164
Missing label for row 20165
Missing label for row 20166
Missing label for row 20167
Missing label for row 20168
Missing label for row 20169
Missing label for row 20170
Missing label for row 20171
Missing label for row 20172
Missing label for row 20173
Missing label for row 20174
Missing label for row 20175
Missing label for row 20176
Missing label for row 20177
Missing label for row 20178
Missing label for row 20179
Missing label for row 20180
Missing label for row 20181
Missing label for row 20182
Missing label for row 20183
Missing label for row 20184
Missing label for ro

Missing label for row 20450
Missing label for row 20451
Missing label for row 20452
Missing label for row 20453
Missing label for row 20454
Missing label for row 20455
Missing label for row 20456
Missing label for row 20457
Missing label for row 20458
Missing label for row 20459
Missing label for row 20460
Missing label for row 20461
Missing label for row 20462
Missing label for row 20463
Missing label for row 20464
Missing label for row 20465
Missing label for row 20466
Missing label for row 20467
Missing label for row 20468
Missing label for row 20469
Missing label for row 20470
Missing label for row 20471
Missing label for row 20472
Missing label for row 20473
Missing label for row 20474
Missing label for row 20475
Missing label for row 20476
Missing label for row 20477
Missing label for row 20478
Missing label for row 20479
Missing label for row 20480
Missing label for row 20481
Missing label for row 20482
Missing label for row 20483
Missing label for row 20484
Missing label for ro

Missing label for row 20754
Missing label for row 20755
Missing label for row 20756
Missing label for row 20757
Missing label for row 20758
Missing label for row 20759
Missing label for row 20760
Missing label for row 20761
Missing label for row 20762
Missing label for row 20763
Missing label for row 20764
Missing label for row 20765
Missing label for row 20766
Missing label for row 20767
Missing label for row 20768
Missing label for row 20769
Missing label for row 20770
Missing label for row 20771
Missing label for row 20772
Missing label for row 20773
Missing label for row 20774
Missing label for row 20775
Missing label for row 20776
Missing label for row 20777
Missing label for row 20778
Missing label for row 20779
Missing label for row 20780
Missing label for row 20781
Missing label for row 20782
Missing label for row 20783
Missing label for row 20784
Missing label for row 20785
Missing label for row 20786
Missing label for row 20787
Missing label for row 20788
Missing label for ro

Missing label for row 21054
Missing label for row 21055
Missing label for row 21056
Missing label for row 21057
Missing label for row 21058
Missing label for row 21059
Missing label for row 21060
Missing label for row 21061
Missing label for row 21062
Missing label for row 21063
Missing label for row 21064
Missing label for row 21065
Missing label for row 21066
Missing label for row 21067
Missing label for row 21068
Missing label for row 21069
Missing label for row 21070
Missing label for row 21071
Missing label for row 21072
Missing label for row 21073
Missing label for row 21074
Missing label for row 21075
Missing label for row 21076
Missing label for row 21077
Missing label for row 21078
Missing label for row 21079
Missing label for row 21080
Missing label for row 21081
Missing label for row 21082
Missing label for row 21083
Missing label for row 21084
Missing label for row 21085
Missing label for row 21086
Missing label for row 21087
Missing label for row 21088
Missing label for ro

In [ ]:
featureArray = np.delete(clean_data_array,missing_labels,axis=0)

In [ ]:
#Normalize the data first
scaler = StandardScaler()
scaler.fit(featureArray[:,1:])
X_train = scaler.transform(featureArray[:,1:])

# Create an SGD regressor
clf = linear_model.SGDRegressor(max_iter = np.ceil(10**6 /X_train.shape[0]),
                                learning_rate = 'optimal',
                                loss = 'squared_loss',
                                penalty = 'l2'
                                )
# Fit the data and the labels
clf.fit(X_train, labels)
print(clf.coef_)